# Installing Libraries & Dependencies

In [ ]:
#!pip3 install -r requirements.txt

In [ ]:
from datetime import datetime
import os

# Input Arguments

Example of input arguments for the data validation component

In [ ]:
PROJECT = "irn-70656-dev-1307100302"
REGION = 'europe-west1'
BUCKET = "bike-sharing-pipeline-metadata"
BUCKET_STAGING = "bike-sharing-pipeline-staging"
PIPELINE_VERSION = "v0_1"
DATA_VERSION = "200909_154702"
MODEL_VERSION = datetime.now().strftime('%y%m%d_%H%M%S')
RUNNER = "AIplatformRunner" # DirectRunner or AIplatformRunner to run on AI platform

In [ ]:
TF_VERSION = '2.2'
PYTHON_VERSION = '3.7'

In [ ]:
# Set up some globals for gcs file
HANDLER = 'gs://' # ../ for local data, gs:// for cloud data

BASE_DIR = os.path.join(HANDLER, BUCKET, PIPELINE_VERSION)
RUN_DIR = os.path.join(BASE_DIR, 'run', DATA_VERSION)

DATA_DIR = os.path.join(RUN_DIR, 'data_transform')
OUTPUT_DIR = os.path.join(RUN_DIR, 'model_training', MODEL_VERSION)


In [ ]:
os.environ['BUCKET'] = BUCKET
os.environ['BUCKET_STAGING'] = BUCKET_STAGING
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['PIPELINE_VERSION'] = PIPELINE_VERSION
os.environ['MODEL_VERSION'] = MODEL_VERSION

In [ ]:
os.environ['HANDLER'] = HANDLER
os.environ['RUNNER'] = RUNNER
os.environ['TF_VERSION'] = TF_VERSION
os.environ['PYTHON_VERSION'] = PYTHON_VERSION

In [ ]:
os.environ['DATA_DIR'] = DATA_DIR
os.environ['OUTPUT_DIR'] = OUTPUT_DIR

### If running on google cloud

Set up project and compute region.

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Move code into python package

Let's package our updated code with feature engineering so it's AI Platform compatible.

In [ ]:
%%bash
mkdir bikesharingmodel
mkdir bikesharingmodel/trainer
touch bikesharingmodel/trainer/__init__.py

In [ ]:
%%writefile bikesharingmodel/PKG-INFO
Metadata-Version: 1.0
Name: trainer
Version: 0.1
Summary: BikeSharingModel
Home-page: UNKNOWN
Author: MA MHANNA
Author-email: maggie.mhanna@renault.com
License: Apache
Description: UNKNOWN
Platform: UNKNOWN

In [ ]:
%%writefile bikesharingmodel/setup.cfg
[egg_info]
tag_build = 
tag_date = 0
tag_svn_revision = 0

In [ ]:
%%writefile bikesharingmodel/setup.py
from setuptools import find_packages, setup

NAME = 'trainer'
VERSION = '0.1'
AUTHOR = 'MA MHANNA'
EMAIL = 'maggie.mhanna@renault.com'

REQUIRED_PACKAGES = ['tensorflow==2.2.0', 
                     'tensorflow-cpu==2.2.0',
                     'tensorflow_transform==0.22.0',
                     'tensorflow_model_analysis==0.22.0', 
                     'apache_beam[gcp]==2.20.0', 
                     'pyarrow==0.16.0',
                     'tfx-bsl==0.22.0',
                     'absl-py==0.8.1']


setup(
    name=NAME,
    version=VERSION,
    author = AUTHOR,
    author_email = EMAIL,
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Bike Sharing Demand Prediction.',
    requires=[]
)

# Create model.py

In [ ]:
%%writefile bikesharingmodel/trainer/model.py

from datetime import datetime
import os
import json
import shutil
import pkg_resources
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

print('INFO: TF version -- {}'.format(tf.__version__))
print('INFO: TFT version -- {}'.format(pkg_resources.get_distribution("tensorflow_transform").version))
print('INFO: TFMA version -- {}'.format(pkg_resources.get_distribution("tensorflow_model_analysis").version))
print('INFO: Beam version -- {}'.format(pkg_resources.get_distribution("apache_beam").version))
print('INFO: Pyarrow version -- {}'.format(pkg_resources.get_distribution("pyarrow").version))
print('INFO: tfx-bsl version -- {}'.format(pkg_resources.get_distribution("tfx-bsl").version))
print('INFO: absl-py version -- {}'.format(pkg_resources.get_distribution("absl-py").version))

# Features, labels, and key columns
NUMERIC_FEATURE_KEYS=["temp", "atemp", "humidity", "windspeed"] 
CATEGORICAL_FEATURE_KEYS=["season", "weather", "daytype"] 
KEY_COLUMN = "datetime"
LABEL_COLUMN = "count"

def transformed_name(key):
    return key 
    
def _get_session_config_from_env_var():
    """Returns a tf.ConfigProto instance that has appropriate device_filters
    set."""

    tf_config = json.loads(os.environ.get('TF_CONFIG', '{}'))

    # Master should only communicate with itself and ps
    if (tf_config and 'task' in tf_config and 'type' in tf_config[
            'task'] and 'index' in tf_config['task']):
        if tf_config['task']['type'] == 'master':
            return tf.ConfigProto(device_filters=['/job:ps', '/job:master'])
        # Worker should only communicate with itself and ps
        elif tf_config['task']['type'] == 'worker':
            return tf.ConfigProto(device_filters=[
                '/job:ps',
                '/job:worker/task:%d' % tf_config['task']['index']
            ])
    return None


def get_dataset_size(file_path):
    """Function that fetchs the size of the Tfrecords dataset."""
    size = 1
    file_list = tf.io.gfile.glob(file_path)
    for file in file_list:
        for record in tf.compat.v1.io.tf_record_iterator(file, options=tf.io.TFRecordOptions(
    compression_type='GZIP')):
            size += 1
    return size
    
# Train and Evaluate input functions
    
def input_fn(data_path, label_column, tf_transform_output, batch_size, mode = tf.estimator.ModeKeys.TRAIN):
    """Create an input function reading TFRecord files using the data API.
    Args:
        data_path: path of the data in tfrecords format
        mode: tf estimator mode key
        batch_size: number of observations in batch

    Returns:
        input_fn: data input function
    """
    
    features_spec = tf_transform_output.transformed_feature_spec()

    def _input_fn():
        # Create list of files in the data path
        file_list = tf.io.gfile.glob(data_path)

        # Create dataset from file list
        dataset = tf.data.TFRecordDataset(filenames=file_list, compression_type = "GZIP", num_parallel_reads=5)
        def parse_example(example):
            parsed_features = tf.io.parse_single_example(example, features_spec)
            label = parsed_features.pop(label_column)
            return parsed_features, label
          
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely, we'll set this in train spec
            dataset = dataset.shuffle(buffer_size=10*batch_size)
        else:
            num_epochs = 1 # end-of-input after one epoch

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.map(parse_example, num_parallel_calls=5)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=1)

    
        return dataset
    
    return _input_fn


# Feature Engineering

def create_feature_columns(tf_transform_output):
    
    numeric_columns = [
      tf.feature_column.numeric_column(transformed_name(key))
      for key in NUMERIC_FEATURE_KEYS
    ]
    
    categorical_columns = [
      tf.feature_column.categorical_column_with_vocabulary_file(
        transformed_name(key), 
        vocabulary_file=tf_transform_output.vocabulary_file_by_name(
            vocab_filename=key), 
        dtype=tf.dtypes.string,
        default_value=None, 
        num_oov_buckets=0)
      for key in CATEGORICAL_FEATURE_KEYS
    ]
    
    indicator_columns = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
    ]
       
    feature_columns = numeric_columns + indicator_columns

    return feature_columns


def rmse(labels, predictions):
    pred_values = predictions['predictions']
    rmse = tf.keras.metrics.RootMeanSquaredError(name="rmse")
    rmse.update_state(y_true=labels, y_pred=pred_values)
    return {'rmse': rmse}

def rmse_2(labels, predictions):
    pred_values = predictions['predictions']
    rmse = tf.compat.v1.metrics.root_mean_squared_error(labels, pred_values)
    return {'rmse': rmse}


def mae(labels, predictions):
    pred_values = tf.squeeze(input = predictions["predictions"], axis = -1)
    mae = tf.keras.metrics.MeanAbsoluteError(name="mae")
    mae.update_state(y_true=labels, y_pred=pred_values)
    return {'mae': mae}


def create_estimator_model(output_dir, feature_columns, hidden_units, run_config):
    model = tf.estimator.DNNRegressor(
        model_dir = output_dir,
        feature_columns = feature_columns,
        hidden_units = hidden_units, # specify neural architecture
        config = run_config
    )
    
    return model

def serving_input_fn(tf_transform_output, label_column):
    """Creates an input function reading from raw data.

    Args:
    tf_transform_output: Wrapper around output of tf.Transform.

    Returns:
    The serving input function.
    """
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label since it is not available during serving.
    raw_feature_spec.pop(label_column)

    def _input_fn():
        """Input function for serving."""
        # Get raw features by generating the basic serving input_fn and calling it.
        # Here we generate an input_fn that expects a parsed Example proto to be fed
        # to the model at serving time.  See also
        # tf.estimator.export.build_raw_serving_input_receiver_fn.
        raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
            raw_feature_spec, default_batch_size=None)
        serving_input_receiver = raw_input_fn()

        # Apply the transform function that was used to generate the materialized
        # data.
        raw_features = serving_input_receiver.features
        transformed_features = tf_transform_output.transform_raw_features(
            raw_features)

        return tf.estimator.export.ServingInputReceiver(
            transformed_features, serving_input_receiver.receiver_tensors)

    return _input_fn

def eval_input_receiver_fn(tf_transform_output, label_column):
    """Function that defines an input placeholder,
     parses and returns features and labels for evaluation."""
    
    def _input_fn():

        serialized_tf_example = tf.compat.v1.placeholder(
            dtype=tf.string, shape=[None], name='input_example_placeholder')

        # This *must* be a dictionary containing a single key 'examples', which
        # points to the input placeholder.
        receiver_tensors = {'examples': serialized_tf_example}

        transformed_feature_spec = tf_transform_output.transformed_feature_spec()
             
        features = tf.io.parse_example(serialized_tf_example, transformed_feature_spec)

        return tfma.export.EvalInputReceiver(
            features=features,
            receiver_tensors=receiver_tensors,
            labels=features[label_column])

    return _input_fn

# train and evaluate

def train_and_evaluate(params):
    
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO) # so loss is printed during training

    # Extract params from task.py
    DATA_DIR = params["data_dir"]
    OUTPUT_DIR = params["output_dir"]
    HIDDEN_UNITS_1 = params["hidden_units_1"]
    HIDDEN_UNITS_2 = params["hidden_units_2"]
    HIDDEN_UNITS_3 = params["hidden_units_3"]
    BATCH_SIZE = params["batch_size"]
    NUM_EPOCHS = params["num_epochs"]
    LEARNING_RATE = params["learning_rate"]

    # Setting up paths 
    TRAIN_PATH = DATA_DIR+'/train*'
    VAL_PATH = DATA_DIR+'/val*'
    TEST_PATH = DATA_DIR+'/test*'
    
    # Training set size
    TRAIN_SIZE = get_dataset_size(TRAIN_PATH)

    NUM_STEPS = TRAIN_SIZE / BATCH_SIZE * NUM_EPOCHS # total steps for which to train model
    CHECKPOINTS_STEPS = 16 # checkpoint every N steps
    
    tf_transform_output = tft.TFTransformOutput(os.path.join(DATA_DIR, 'tft_output'))

    FEATURE_COLUMNS = create_feature_columns(tf_transform_output)
    
    run_config = tf.estimator.RunConfig(
        tf_random_seed = 1, # for reproducibility
        save_checkpoints_steps = CHECKPOINTS_STEPS, # checkpoint every N steps
        save_summary_steps = int(CHECKPOINTS_STEPS/5),
        session_config = _get_session_config_from_env_var()
        )
    
    estimator = create_estimator_model(OUTPUT_DIR, FEATURE_COLUMNS, [HIDDEN_UNITS_1, 
                                       HIDDEN_UNITS_2, HIDDEN_UNITS_3], run_config)
    
    estimator = tf.estimator.add_metrics(estimator = estimator, metric_fn = rmse) 
    estimator = tf.estimator.add_metrics(estimator = estimator, metric_fn = mae) 

    train_input_fn = input_fn(TRAIN_PATH, LABEL_COLUMN, tf_transform_output, 
                              BATCH_SIZE, tf.estimator.ModeKeys.TRAIN)
    val_input_fn = input_fn(VAL_PATH, LABEL_COLUMN, tf_transform_output, 
                            BATCH_SIZE, tf.estimator.ModeKeys.EVAL)
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = train_input_fn,
        max_steps = NUM_STEPS)

    exporter = tf.estimator.LatestExporter(name = 'exporter', 
               serving_input_receiver_fn = serving_input_fn(tf_transform_output, LABEL_COLUMN))

    eval_spec = tf.estimator.EvalSpec(
        input_fn = val_input_fn,
        steps = CHECKPOINTS_STEPS, # Number of steps to run evalution for at each checkpoint
        start_delay_secs = 1, # wait at least N seconds before first evaluation (default 120)
        throttle_secs = 16, # wait at least N seconds before each subsequent evaluation (default 600)
        exporters = exporter) # export SavedModel once at the end of training
    
    tf.estimator.train_and_evaluate(
        estimator = estimator, 
        train_spec = train_spec, 
        eval_spec = eval_spec) 
    
    # Also export the EvalSavedModel
    tfma.export.export_eval_savedmodel(
        estimator=estimator, export_dir_base=OUTPUT_DIR + '/eval_saved_model/',
        eval_input_receiver_fn=eval_input_receiver_fn(tf_transform_output, LABEL_COLUMN))
    

# Create task.py

In [ ]:
%%writefile bikesharingmodel/trainer/task.py
import argparse
import json
import os

from . import model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_dir",
        help = "Location directory to read datasets",
        type = str,
        required = True
    )
    parser.add_argument(
        "--output_dir",
        help = "Location directory to write checkpoints and export models",
        type = str,
        required = True
    )
    parser.add_argument(
        "--hidden_units_1",
        help = "Hidden layer 1 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--hidden_units_2",
        help = "Hidden layer 2 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--hidden_units_3",
        help = "Hidden layer 3 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--batch_size",
        help = "Mini-batch gradient descent batch size",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--num_epochs",
        help = "Number of epochs",
        type = int,
        default = 10
    )
    parser.add_argument(
        "--learning_rate",
        help = "Hidden layer 1 sizes to use for DNN feature columns",
        type = int,
        default = 0.001
    )
    args = parser.parse_args().__dict__


    # Append trial_id to path so trials don't overwrite each other
    args["output_dir"] = os.path.join(
        args["output_dir"],
        json.loads(
            os.environ.get("TF_CONFIG", "{}")
        ).get("task", {}).get("trial", "")
    ) 
    
    # Run the training job
    model.train_and_evaluate(args)

# Run locally

In [ ]:
%%bash
if [ "$RUNNER" = "DirectRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi  

    python -m bikesharingmodel.trainer.task \
        --data_dir=${DATA_DIR} \
        --output_dir=${OUTPUT_DIR}
else 
echo "RUNNER = " $RUNNER
fi

# Running locally using gcloud

In [ ]:
%%bash
if [ "$RUNNER" = "DirectRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    # Use AI Platform to train the model in local file system
    gcloud ai-platform local train \
        --module-name=trainer.task \
        --package-path=bikesharingmodel/trainer \
        -- \
        --data_dir=${DATA_DIR} \
        --output_dir=${OUTPUT_DIR}
else 
echo "RUNNER = " $RUNNER
fi

# TensorBoard 

In [ ]:
%load_ext tensorboard

In [ ]:
#%tensorboard --logdir $OUTPUT_DIR

# Run on AI Platform using gcloud

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-standard-4
  evaluatorType: n1-standard-4
  evaluatorCount: 1

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}
    echo $OUTPUT_DIR $REGION $JOB_NAME

    gcloud ai-platform jobs submit training $JOB_NAME \
       --region=$REGION \
       --module-name=trainer.task \
       --package-path=bikesharingmodel/trainer \
       --staging-bucket=gs://$BUCKET_STAGING \
       --config=config.yaml \
       --runtime-version=$TF_VERSION \
       --python-version=$PYTHON_VERSION \
       -- \
       --data_dir=$DATA_DIR \
       --output_dir=$OUTPUT_DIR 
else 
echo "RUNNER = " $RUNNER
fi

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then
    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}
    gcloud ai-platform jobs describe $JOB_NAME
fi

# Run on AI Platform with Hyperparameter Tuning

AI Platform HyperTune, powered by Google Vizier, uses Bayesian Optimization by default, but also supports Grid Search and Random Search.

When tuning just a few hyperparameters (say less than 4), Grid Search and Random Search work well, but when tunining several hyperparameters and the search space is large Bayesian Optimization is best.

In [ ]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-standard-4
  evaluatorType: n1-standard-4
  evaluatorCount: 1
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 4
    maxParallelTrials: 2
    hyperparameterMetricTag: rmse
    enableTrialEarlyStopping: True   
    params:
    - parameterName: hidden_units_1
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units_2
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units_3
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}_hypertune
    echo $OUTPUT_DIR $REGION $JOBNAME

    gcloud ai-platform jobs submit training $JOB_NAME \
       --region=$REGION \
       --module-name=trainer.task \
       --package-path=bikesharingmodel/trainer \
       --staging-bucket=gs://$BUCKET_STAGING \
       --runtime-version=$TF_VERSION \
       --python-version=$PYTHON_VERSION \
       --config=hyperparam.yaml \
       -- \
       --data_dir=$DATA_DIR \
       --output_dir=$OUTPUT_DIR \
       --num_epochs=10  

else 
echo "RUNNER = " $RUNNER
fi

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then
    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}_hypertune
    gcloud ai-platform jobs describe $JOB_NAME
fi

Best hyperparameter trial:

In [ ]:
%%bash
JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}_hypertune
TRIAL_ID=$(gcloud ai-platform jobs describe $JOB_NAME --format 'value(trainingOutput.trials.trialId.slice(0))')
echo $TRIAL_ID